# End of Week 1 - Exercise - Using Gemini API with GenAI SDK

In [ ]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types
from IPython.display import Markdown, display, update_display

In [ ]:
# Load API Key - For Gemini it automatically takes the api key from env file if we save the key using GOOGLE_API_KEY keyword
load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
# Initialze Google Client
# Just to make it explicit i have used the api_key parameter but thats optional and genai.client automatically takes from .env file

try:
    client = genai.Client(api_key=api_key)
    print("Google GenAI Client initialized successfully!")
except Exception as e:
    print(f"Error initializing GenAI Client: {e}")
    print("Ensure your GOOGLE_API_KEY is correctly set as an environment variable.")
    exit()

In [ ]:
# Get list of models
print("List of models that support generateContent:\n")
for m in client.models.list():
    for action in m.supported_actions:
        if action == "generateContent":
            print(m.name)

In [ ]:
MODEL_GEMINI = 'gemini-2.5-flash-preview-05-20'

In [ ]:
# System Definitions

system_instruction_prompt = (
    "You are an expert Python programming assistant. Your goal is to identify common coding errors, suggest improvements for readability and efficiency,and provide corrected code snippets.\
    Always format code blocks using Markdown.\
    Be concise but thorough. Focus on the provided code and context."
)

generate_content_config = types.GenerateContentConfig(system_instruction=system_instruction_prompt)

In [ ]:
# Main program to get user input and then use model to respond.

MAX_HISTORY_MESSAGES = 6
conversation_contents = []

print("\n--- Start Chat with Gemini Python Assistant ---")
print("Type 'Done' to exit the conversation.")

while True:
    user_input = input("You: ").strip()

    if user_input.lower() == "done": 
        print("\nExiting chat. Goodbye!")
        break 

    if not user_input: 
        print("Please enter a question or 'Done' to exit.")
        continue
    
    try:
        user_message_content = types.Content(
            role="user",
            parts=[types.Part.from_text(text=user_input)]
        )  
        
        conversation_contents.append(user_message_content)    
        
        stream_response = client.models.generate_content_stream(
            model=MODEL_GEMINI,
            contents=conversation_contents,
            config=generate_content_config,
        )
            
        model_full_response_text = "**Gemini:**\n\n"
        current_display_handle = display(Markdown(""), display_id=True)
      
            
        for chunk in stream_response:
            chunk_text = chunk.text or ''
            model_full_response_text += chunk_text
            update_display(Markdown(model_full_response_text), display_id=current_display_handle.display_id)
        
        # Add Model's FULL Response to Conversation History
        model_message_content = types.Content(
            role="model",
            parts=[types.Part.from_text(text=model_full_response_text.removeprefix("**Gemini:**\n\n"))]
        )
        
        conversation_contents.append(model_message_content)
        
        conversation_contents = conversation_contents[-MAX_HISTORY_MESSAGES:]    

    except Exception as e:
        print(f"\nAn error occurred during interaction: {e}")
        if conversation_contents:
            conversation_contents.pop()
        print("Please try asking your question again or type 'Done' to exit.")
        continue  